In [1]:
import os 
import pandas as pd
from pathlib import Path
import sys
import numpy 
from typing import List, Dict, Any, Optional
from charset_normalizer import from_path 

MAIN_PATH = Path().resolve().parent
sys.path.append(str(MAIN_PATH))

from src.utilities import load_config, load_validations, load_fields, load_data, init_log, format_file_size

import src.analisys 
from src.analisys import field_apply_list, check_null_empty, check_values_list, check_regex_format, check_zero_values, check_negative_values, check_valid_range

from src.utilities.logger import log_event

In [2]:
# Carrega as configurações e inicia o log 
df_config = pd.DataFrame([{}])
load_config(df_config)

# inicia o log com o path parametrizado
init_log(df_config.loc[0, "log_path"])

# Carrega a lista de validações
df_validations = pd.DataFrame([{}])
load_validations(df_validations,df_config.loc[0, "eda_config_path"] )

# Carrega a lista de campos a validar
df_fields = pd.DataFrame([{}])
load_fields(df_fields, df_config.loc[0, "eda_config_path"])

,file,table,field,type,subtype,null,zero,negative,pk,fk,table_fk,format,format_regex,range,values,null_limit,active
0,ses_seguros.csv,ses_seguros,coenti,number,integer,no,no,no,yes,yes,ses_cias,NaN,NaN,NaN,NaN,NaN,yes
1,ses_seguros.csv,ses_seguros,damesano,data,undefined,no,no,no,yes,no,NaN,YYYYMM,^(199[0-9]|20[0-4][0-9]|2050)(0[1-9]|1[0-2])$,NaN,NaN,NaN,yes
2,ses_cias.csv,ses_cias,coenti,number,integer,no,no,no,yes,no,NaN,NaN,NaN,de 1111 a 99999,123.0,NaN,yes
3,ses_ramos.csv,ses_ramos,coramo,number,integer,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,yes
4,ses_ramos.csv,ses_ramos,banana,number,integer,no,no,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,yes


In [3]:
result: List[Dict[str, Any]] = []
# rotina pra salvar os resutlados das analises 
def save_result(
    file: str, field: str, category: str, test: str, evidence: Any, detail: Optional[str] = None, status: str = "PASS") -> result:    
    registry: result = { "file": file, "Field": field, "category": category, "test": test, "evidence": evidence, "detail": detail, "status": status,}
    return registry

Inicializa variaveis para leitura dos campos configurados para analise 

In [4]:
df_data = pd.DataFrame([{}])
FAIL_TABLES = set()   # Controla falhas estruturais (Passo 1)
loaded_file = ""
separator = df_config.loc[0, "separator"]
encode = df_config.loc[0, "encode"]
new_file = True 
RESULTS = [] 

# Converte todos os campos para str
df_fields = df_fields.astype('string')

# Deixa todos os nomes de campos em minusculo e remove espaços
df_fields.columns = df_fields.columns.str.strip().str.lower()

Inicio do Loop no campo a serem analisados

In [5]:
for index, row in df_fields.iterrows():

    file_name = row["file"]
    table_name = row["table"]
    file_path = df_config.loc[0, "data_path"] + file_name

    # Verifica se o arquivo está carregado ou já falhou
    if (file_name in FAIL_TABLES): 
        continue 
    
    if (row["file"] != loaded_file):       
        try: 
            try: # Carregamento do arquivo de dados

                # Detecta o encode, se não foi fornecido
                try:
                    if len(encode) == 0: 
                        detectado = from_path(str(file_path)).best() 
                        encoding_detectado = detectado.encoding 
                    else: 
                        encoding_detectado = encode
                except Exception:
                    encoding_detectado = 'utf-8' 

                # data Load    
                df_data = pd.read_csv(file_path,encoding=encoding_detectado,sep=separator,engine='python')

            except Exception as e:
                raise ValueError(f"Falha no carregamento do arquivo !")

            loaded_file = file_name
            new_file = True  
        except UnicodeDecodeError:
            print(f"❌ Erro de Codificação: O arquivo pode não ser '{encoding}'. Tente outro encoding.")
        
        except pd.errors.ParserError as e:
            # Este erro geralmente indica problemas de estrutura (delimitadores incorretos, linhas mal formadas)
            print(f"❌ Erro de Parsing (Estrutura CSV incorreta): {e}")
        
        except Exception as e:
            # Captura outros erros, como FileNotFoundError
            print(f"❌ Erro inesperado ao ler o arquivo: {e}") 

    if new_file: 
        print("-" * 30 )
        print(table_name)
        print("-" * 30 )
        # Corrige o tipo das colunas "int" exibidas como "float" 
        for col in df_data.select_dtypes(include=["float"]).columns:
            if (df_data[col].dropna() % 1 == 0).all():
                df_data[col] = df_data[col].astype("Int64")        

        # Coleta estatisticas no nivel do arquivo 
        file_size = format_file_size(os.path.getsize(file_path) )
        line_count = len(df_data)
        col_count = len(df_data.columns) 
        evidence_str = "Tamanho: " + file_size + " Linhas/Colunas: " + str(line_count) + "/" + str(col_count)
        RESULTS.append(save_result(row["file"],  "Todos","structure","file info",evidence_str,"", "pass")) 

        # Sanitização da taberla de dados 
        # Deixa todos os nomes de campos em minusculo e remove espaços
        df_data.columns = df_data.columns.str.strip().str.lower()   
        # print(df_data.dtypes)



        # Testa colunas faltantes
        missing_columns_lst = [] 
        df_expected_columns = (df_fields[df_fields['table'] == table_name]['field'].unique()) 
        data_column_names = df_data.columns.to_list()
        df_data_columns = pd.DataFrame({"column": data_column_names})
        expected_set = set([c.strip().lower() for c in df_expected_columns])
        data_set = set(df_data_columns['column'].str.strip().str.lower().tolist())
        missing_colunms = expected_set.difference(data_set)
        missing_columns_lst = list(missing_colunms)
        if missing_colunms:
            str_missing_columns = ", ".join(sorted(list(missing_colunms)))
        else:
            str_missing_columns = "nenhuma"
        
        # Testa colunas com nome duplicados
        nomes_colunas = [col.strip().lower() for col in df_data.columns.tolist()]
        nomes_series = pd.Series(nomes_colunas)
        nomes_sem_sufixo = nomes_series.str.replace(r'\.\d+$', '', regex=True)
        contagem_nomes = nomes_sem_sufixo.value_counts()
        nomes_duplicados_series = contagem_nomes[contagem_nomes > 1]
        lista_nomes_duplicados = nomes_duplicados_series.index.tolist()
        colunas_duplicadas = df_data_columns['column'].value_counts()
        if lista_nomes_duplicados:
            # Junta os nomes duplicados com ", "
            resultado_string = ", ".join(lista_nomes_duplicados)
        else:
            resultado_string = "nenhum"
        if resultado_string == "nenhum" and str_missing_columns == "nenhuma": 
            status = "pass" 
        else: 
            status = "fail"

        evidence_str =  "Faltantes: " +  str_missing_columns + " Nomes_duplicados: " + resultado_string          
        RESULTS.append(save_result(row["file"], "Todos" ,"structure","Column info",evidence_str,"", status)) 
            
        new_file = False

    print("")
    print("***********  " + row["field"] + "  **************")

    # Monta a lista de caracteristicas do campo para chamada dos checagens corresposndentes 
    apply_list = field_apply_list(df_data, df_fields, row)
    print("---- Caracteristicas ----")
    for item in  apply_list: 
        print(item) 

    # Busca as checagens Ativas correspondentes as caracteristicas do campo  
    apply_set = set([item.strip().lower() for item in apply_list]) 
    mask_apply = df_validations['apply'].apply(lambda x: len(set(str(x).lower().replace(' ', '').split(',')) & apply_set) > 0)
    mask_active = (df_validations['active'] == 'yes')
    df_filtrado = df_validations[mask_apply&mask_active]

    # Loop de chamada das analises 
    print("----- Analises aplicadas ----")
    for index, check_row in df_filtrado.iterrows():
        try: 
            print("Teste: " + check_row["test"]+ " - Apply: " + check_row["apply"] ) 
            if str(row["field"]) in list(missing_columns_lst): # ignora colunas que não foram encontrados
                continue  
            #  chama a rotina parametrizada para a analise
            evidence, status, detail  = getattr(src.analisys,check_row["routine"])(df_data, df_fields, row)  
            # Salva o resultado das analise no relatõrio final
            RESULTS.append(save_result(row["file"], row["field"] ,check_row["category"],check_row["test"],evidence,detail, status)) 
        except Exception as e:
            evidence = "Falha na chamada da rotina de analise"
            status = "error"
            detail = (f"Rotina '{check_row["routine"]}': " f"{type(e).__name__}: {str(e)}")
            RESULTS.append(save_result(row["file"], row["field"] ,check_row["category"],check_row["test"],evidence,detail, status)) 

            continue

    # Chama as rotinas basicas (qualquer tipo de campo)
    # df_basic_checks = df_validations[(df_validations['category'] == 'basic') & (df_validations['active'] == 'yes')]
    # print("df_basic_checks")
    # print(df_basic_checks)
    # for index, check_row in df_basic_checks.iterrows():
    #     evidence, status, detail  = getattr(src.analisys,check_row["routine"])(df_data, df_fields, row)
    #     print("Retorno")
    #     print(evidence, status, detail)

        



------------------------------
ses_seguros
------------------------------

***********  coenti  **************
---- Caracteristicas ----
number
integer
no-null
pk
fk
no-zero
no-negative
----- Analises aplicadas ----
Teste: Null or empty values - Apply: no-null
Teste: Zeros values - Apply: no-zero
-------- Debug ----------
2100
100
0
-------- Debug ----------
-------- Debug df_valores_numericos----------
coenti
int64
-------- Debug ----------
-------- Debug series_alvo ----------
int64
-------- Debug ----------
Teste: Negative values - Apply: no-negative

***********  damesano  **************
---- Caracteristicas ----
data
no-null
pk
no-zero
no-negative
format
----- Analises aplicadas ----
Teste: Null or empty values - Apply: no-null
Teste: Expected format (regex) - Apply: format
Teste: Zeros values - Apply: no-zero
-------- Debug ----------
2100
100
0
-------- Debug ----------
-------- Debug df_valores_numericos----------
damesano
int64
-------- Debug ----------
-------- Debug series_a

Fim do loop de analise e de campos

Gera o relatõrio de saida

In [6]:
"""
    Exibe os registros de auditoria em um formato tabular otimizado
    para visualização em tela, sem quebras de linha em um mesmo registro.
    """
# 1. Manter a configuração do cabeçalho à esquerda (que você já fez)
pd.set_option('display.colheader_justify', 'left') 
pd.set_option('display.max_colwidth', None)

# 2. Definir a ordem das colunas
colunas_exibicao = ['status', 'file', 'Field', 'test', 'evidence', 'detail']

if RESULTS:
    DF_RESULTS = pd.DataFrame(RESULTS)

    # --- A CORREÇÃO CRÍTICA ---
    # Usar justify='left' no método to_string() força TODO o conteúdo
    # (incluindo números) a ser alinhado à esquerda.
    tabela_formatada = DF_RESULTS[colunas_exibicao].to_string(justify='left')
    
    # 3. Imprimir
    print("\n" + "="*80)
    print("--- 📋 REGISTROS DE AUDITORIA ---".center(80))
    print("="*80)
    print(tabela_formatada)
    
    # 4. Reverter as opções (boa prática)
    pd.reset_option('display.colheader_justify')
    pd.reset_option('display.max_colwidth')

else:
    print("A lista 'RESULTS' está vazia.")


                        --- 📋 REGISTROS DE AUDITORIA ---                        
   status file             Field     test                     evidence                                     detail                                                         
0   pass   ses_seguros.csv     Todos                file info      Tamanho: 7.56 KB Linhas/Colunas: 100/21                                                                
1   pass   ses_seguros.csv     Todos              Column info  Faltantes: nenhuma Nomes_duplicados: nenhum                                                                
2   pass   ses_seguros.csv    coenti     Null or empty values                                 0 ausências.               Nenhum valor nulo ou vazio detectado neste campo.
3   fail   ses_seguros.csv    coenti             Zeros values                               Zerados: 1.00%            Linha com exemplo de erro: (70): Valor encontrado: 0
4   fail   ses_seguros.csv    coenti          Negative values  